In [1]:
import pandas as pd

In [56]:
#load data
rating_data= pd.read_csv('archive/ratings.csv')
movie_data = pd.read_csv('archive/movies_metadata.csv')

C:\Users\KTDS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
rating_data.drop('timestamp',axis = 1,inplace = True)


In [58]:
rating_data['id'] = rating_data['movieId']
rating_data.drop('movieId',axis = 1, inplace = True)

In [59]:
def index_strange_id(df):
    index = []
    for idx in range(df.shape[0]):
        try:
            pd.to_numeric(df['id'][idx])
        except ValueError:
            index.append(idx)
        
    return index

In [60]:
strange_index = index_strange_id(movie_data)

movie_data.drop(index = strange_index, axis = 0,inplace = True)
movie_data['id'] = pd.to_numeric(movie_data['id'])

In [65]:
movies = movie_data[["id","genres","title"]]

In [66]:

user_movie_rating = pd.merge(rating_data,movies, on = 'id')

In [73]:
from ast import literal_eval
def safe_literal_eval(node):
    try:
        return literal_eval(node)
    except ValueError:
        print(node)
        return node 

In [82]:
sampling = user_movie_rating[:150000]
#데이터가 너무 많으므로 150000개만 sampling 하여 진행
#원래는 sampling 자체를 title column이 분보도가 비슷하게 해야 하지만 그냥 진행

In [84]:
sampling['final_genres'] = sampling['genres'].apply(
    lambda x: safe_literal_eval(x)
)

<ipython-input-84-ab6895e065ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling['final_genres'] = sampling['genres'].apply(


In [87]:
sampling['final_genres'] = sampling['final_genres'].apply(
    lambda x : [d['name'] for d in x]
)

<ipython-input-87-1f91034adee2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling['final_genres'] = sampling['final_genres'].apply(


In [91]:
sampling['final_genres'] = sampling['final_genres'].apply(
    lambda x : x[0]
)

<ipython-input-91-d20c50f44725>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling['final_genres'] = sampling['final_genres'].apply(


In [96]:
sampling.drop("genres",axis = 1,inplace = True)

C:\Users\KTDS\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [98]:
m_u_rating = sampling.pivot_table('rating',index = 'title',columns = 'userId')

In [99]:
m_u_rating

userId,1,3,5,9,11,12,20,22,24,27,...,270882,270884,270885,270887,270891,270892,270893,270894,270895,270896
title,,,,,,,,,,,,,,,,,,,,,
Rocky Balboa,5.0,NaN,NaN,4.5,NaN,5.0,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sleepless in Seattle,5.0,4.0,5.0,NaN,NaN,4.0,4.5,NaN,5.0,5.0,...,NaN,4.0,5.0,5.0,NaN,4.5,NaN,4.0,NaN,5.0
The 400 Blows,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Three Colors: Red,1.0,NaN,NaN,NaN,3.5,NaN,NaN,5.0,5.0,NaN,...,5.0,NaN,4.5,NaN,5.0,NaN,5.0,NaN,4.0,5.0


In [100]:
u_m_rating = sampling.pivot_table('rating',index = 'userId',columns = 'title')

In [101]:
u_m_rating

title,Rocky Balboa,Sleepless in Seattle,The 400 Blows,Three Colors: Red
userId,,,,
1,5.0,5.0,4.5,1.0
3,NaN,4.0,NaN,NaN
5,NaN,5.0,NaN,NaN
9,4.5,NaN,NaN,NaN
11,NaN,NaN,NaN,3.5
...,...,...,...,...
270892,NaN,4.5,NaN,NaN
270893,NaN,NaN,NaN,5.0
270894,NaN,4.0,NaN,NaN


In [102]:
m_u_rating.fillna(0,inplace = True)

In [105]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [106]:
item_based_collabor = cosine_similarity(m_u_rating)

In [107]:
item_based_collabor

array([[1.        , 0.32364098, 0.1497164 , 0.32298149],
       [0.32364098, 1.        , 0.12673735, 0.41003105],
       [0.1497164 , 0.12673735, 1.        , 0.15418458],
       [0.32298149, 0.41003105, 0.15418458, 1.        ]])

In [108]:
item_based_colla = pd.DataFrame(data = item_based_collabor, index = m_u_rating.index,columns = m_u_rating.index)

In [113]:
item_based_colla

title,Rocky Balboa,Sleepless in Seattle,The 400 Blows,Three Colors: Red
title,,,,
Rocky Balboa,1.000000,0.323641,0.149716,0.322981
Sleepless in Seattle,0.323641,1.000000,0.126737,0.410031
The 400 Blows,0.149716,0.126737,1.000000,0.154185
Three Colors: Red,0.322981,0.410031,0.154185,1.000000


In [120]:
def recommend(title):
    return item_based_colla[title].sort_values(ascending = False)[1:]

In [121]:
recommend('Rocky Balboa')

title
Sleepless in Seattle    0.323641
Three Colors: Red       0.322981
The 400 Blows           0.149716
Name: Rocky Balboa, dtype: float64